In [1]:
import requests
requests.packages.urllib3.disable_warnings()
import json
import numpy as np

In [2]:
def encodeData(data: np.ndarray) -> bytes:
    """
    Encodes the data. Returs the encoded data in bytes to be decoded in the handler.
    Consists of:
        3 bytes (int-big) with the length of the metadata
        The corresponding amount of utf-8 bytes with the stringified json metadata
        The numpy array as bytes

    """

    jsonMetadata = {
        "shape": list(data.shape)
    }

    metadataBytes = str(json.dumps(jsonMetadata)).encode("utf-8")
    metadataLength = len(metadataBytes).to_bytes(3)

    return metadataLength + metadataBytes + data.tobytes()

def decodeData(data: bytes) -> [dict, np.ndarray]:
    """
    Decodes the data coming from torchserve. Response is structured the same way as above
    
    """
    metadataLength = int.from_bytes(data[:3])

    rawMetadata = data[3:metadataLength+3]
    metadata = json.loads(rawMetadata.decode('utf-8'))

    array = np.frombuffer(data[metadataLength+3:], dtype=np.float32).reshape(metadata["shape"])
    return (metadata, array)


In [3]:
def makeRequest(dataArray: np.ndarray, modelName:str = "model", port: int = 8443) -> requests.Response:
    """
    Mock request.
    
    """

    url = f"https://127.0.0.1:{port}/predictions/{modelName}"

    r = requests.get(url, data=encodeData(dataArray), verify=False)
    return r

In [4]:
data = np.random.rand(3,2)
r = makeRequest(data)

In [5]:
(metadata, array) = decodeData(r.content)

print(metadata, array)

{'shape': [3, 2]} [[0.5352362  0.8371547 ]
 [0.5346536  0.8348037 ]
 [0.535901   0.83983785]]
